# Capstone Project: Vital Statistics of Professional Athletes

## Problem Statement:
### What impact does a career in professional sports have on the life expectancy of an athlete?

More specifically:
- Do different sports have different levels of impact on athletes' life expectancies?
- Are life expectancies further affected by the length of the playing career, the number of games played, or the positions they played?
- Can I build a model that predicts how many players will die in a given year, as well as the distribution of their ages when they died? Would this model be better than one based on vital statistics of the general population?

### Proposed Data Sources:

#### Athlete Statistics

- baseball-reference.com
- pro-football-reference.com
- hockey-reference.com

The Baseball and Football reference sites have individual pages that list all the players that died in a given year. They also list the length of their careers, when they played, and the total number of games played. The football site lists the players' positions, while the baseball site merely separates batters from pitchers. The football site also lists coaches along with players.

The hockey reference site doesn't list players who died in a certain year, but they do list all the players born in a given year. These listings also contain the death date of each player. However, the death dates of some players may be unknown.

There is a basketball reference site that has each player's death dates only on their individual pages. There are listings of players who were born in a given year, but the death dates aren't listed on those pages. Scraping for death dates might take considerably more work.

If there is time, I would like to see if I could collect data on additional sports such as soccer, tennis, golf, boxing, auto racing, or cycling.

#### General Statistics

An example of where I can get the age distribution of all deaths that happened in the U.S. is 
https://www.statista.com/statistics/241572/death-rate-by-age-and-sex-in-the-us/